In [1]:
import pandas as pd

import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="gcp/cms-transparency-3f221fdca717.json"

In [2]:
data = pd.read_csv('BeerDataScienceProject.csv', encoding='latin-1')
data.sample(5)

,beer_ABV,beer_beerId,beer_brewerId,beer_name,beer_style,review_appearance,review_palette,review_overall,review_taste,review_profileName,review_aroma,review_text,review_time
345253,6.9,129,37,Orval Trappist Ale,Belgian Pale Ale,4.5,3.5,3.5,4.5,rapidsequence,4.0,Poured from a twelve oz bottle (maybe 11.2-- i...,1244078113
470632,7.0,16637,392,Black Hole,Old Ale,3.0,3.5,4.0,4.0,matty,4.0,Pitch black beer with slowly fading rich foamy...,1231827619
81516,5.0,3558,140,Sierra Nevada Summerfest Lager,Czech Pilsener,3.0,2.5,3.0,3.0,changeup45,3.0,My first reaction to this was that it tasted l...,1219530069
390080,5.0,35134,3,Pecan Harvest Ale,American Brown Ale,4.0,3.0,3.5,3.5,farrago,3.0,"Solid two fingers of cream white foam, good de...",1287111471
388274,5.8,48775,17893,Rebel Ale,American Amber / Red Ale,3.5,3.0,4.0,3.5,CpnYates,2.5,"Purchased at the superb vintage cellar, Blacks...",1257196931


In [3]:
data.drop_duplicates(subset=['beer_beerId', 'beer_brewerId', 'review_profileName'], keep='last', inplace=True)
data.isna().sum()/len(data) * 100

beer_ABV              3.848172
beer_beerId           0.000000
beer_brewerId         0.000000
beer_name             0.000000
beer_style            0.000000
review_appearance     0.000000
review_palette        0.000000
review_overall        0.000000
review_taste          0.000000
review_profileName    0.020989
review_aroma          0.000000
review_text           0.022515
review_time           0.000000
dtype: float64

In [4]:
df = data.dropna(axis=0, subset=['review_text'])

In [5]:
from google.cloud import language_v1

def gcp_nlp_analyze_sentiment(text_content):
    """
    Analyzing Sentiment in a String

    Args:
      text_content The text content to analyze
    """

    client = language_v1.LanguageServiceClient()
    type_ = language_v1.Document.Type.PLAIN_TEXT
    language = "en"
    document = {"content": text_content, "type_": type_, "language": language}
    encoding_type = language_v1.EncodingType.UTF8
    response = client.analyze_sentiment(request = {'document': document, 'encoding_type': encoding_type})
    return response.document_sentiment.score

In [ ]:
df['sentiment_score'] = df['review_text'].apply(gcp_nlp_analyze_sentiment)
df.to_csv('sentiment_scores.csv')